In [1]:
import pandas as pd
from nxontology.imports import from_file
import time
import torch

nxo = from_file('/home/elliot/Documents/ESL2024/data/go-basic.json.gz')
nxo.freeze()

file_path = "/home/elliot/Documents/ESL2024/data/little_iric.tsv"


In [2]:
iric = pd.read_csv(file_path, delimiter='\t', names = ['subject', 'predicate','object'])
display(iric)

# Mapping entities to ids

entity_set = set(iric['object']).union(set(iric['subject']))
entity_to_mapping = {entity: int(i) for i, entity in enumerate(entity_set)}
relation_set = set(iric['predicate'])
relation_to_mapping = {relation: int(i) for i, relation in enumerate(relation_set)}

mapped_iric = iric.copy()
mapped_iric['object'] = mapped_iric['object'].apply(lambda x: entity_to_mapping[x])
mapped_iric['subject'] = mapped_iric['subject'].apply(lambda x: entity_to_mapping[x])
mapped_iric['predicate'] = mapped_iric['predicate'].apply(lambda x: relation_to_mapping[x])

# display(mapped_iric)

mapping_to_entity = {v: k for k, v in entity_to_mapping.items()}
mapping_to_relation = {v: k for k, v in relation_to_mapping.items()}

,subject,predicate,object
0,GO:0000001,is_a,GO:0048311
1,GO:0000001,is_a,GO:0048308
2,GO:0000002,is_a,GO:0007005
3,GO:0000003,is_a,GO:0008150
4,GO:0000006,is_a,GO:0005385
...,...,...,...
9995,GO:0009920,is_a,GO:1903047
9996,GO:0009921,is_a,GO:0098797
9997,GO:0009922,is_a,GO:0016747
9998,GO:0009923,is_a,GO:1990234


In [3]:
shuff_iric = iric.copy()
shuff_iric['object'] = shuff_iric['object'].sample(frac=1).reset_index(drop=True)
display(shuff_iric)

,subject,predicate,object
0,GO:0000001,is_a,GO:0051326
1,GO:0000001,is_a,GO:0016841
2,GO:0000002,is_a,GO:0033993
3,GO:0000003,is_a,GO:0003018
4,GO:0000006,is_a,GO:0009185
...,...,...,...
9995,GO:0009920,is_a,GO:0002604
9996,GO:0009921,is_a,GO:1905202
9997,GO:0009922,is_a,GO:0009368
9998,GO:0009923,is_a,GO:0016836


In [28]:
def lin_sim(n1,n2, ontology):
    try:
        return ontology.similarity(n1, n2).lin
    except Exception as e:
        # print(e)
        return 0
def lin_sim_on_row(row):
    return lin_sim(row['subject'], row['object'], nxo)

In [34]:
%%timeit
shuff_iric["LinSim"] = shuff_iric.swifter.apply(func=lin_sim_on_row, axis=1,)

Pandas Apply: 100%|██████████| 10000/10000 [00:00<00:00, 56843.55it/s]

211 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
display(shuff_iric)

In [36]:
%%timeit
shuff_iric["LinSim"] = shuff_iric.apply(func=lin_sim_on_row,
                            axis=1,
                            )

161 ms ± 2.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
display(shuff_iric)

,subject,predicate,object,LinSim
0,GO:0000001,is_a,GO:0051326,0.062571
1,GO:0000001,is_a,GO:0016841,0.000000
2,GO:0000002,is_a,GO:0033993,0.070031
3,GO:0000003,is_a,GO:0003018,0.079098
4,GO:0000006,is_a,GO:0009185,0.000000
...,...,...,...,...
9995,GO:0009920,is_a,GO:0002604,0.062497
9996,GO:0009921,is_a,GO:1905202,0.343621
9997,GO:0009922,is_a,GO:0009368,0.000000
9998,GO:0009923,is_a,GO:0016836,0.000000


In [37]:
%%timeit
# FOR
for index, row in iric.iterrows():
    row['Sim to GO:0048308'] = lin_sim(row['object'], 'GO:0048308', nxo)


3.58 s ± 53.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
a = torch.Tensor([0.23231,0.23321])
b = torch.Tensor([0.23231,0.23321])
c = torch.Tensor([0.23231,0.23321])
d = torch.Tensor([0.23231,0.23321])


139781948553232

In [38]:
hash(a)

139781948553232